# QueryProcessor Exploration: No chat history, gpt-4o

In [1]:
import os
import sys

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

import asyncio
from typing import Dict, List
from models.message import Message
from models.product import Product
from services.weaviate_service import WeaviateService
from services.openai_service import OpenAIService
from services.query_processor import QueryProcessor
from config import Config

# Load configuration
config = Config()

# Initialize services
weaviate_service = WeaviateService()
await weaviate_service.initialize_weaviate(
        config.OPENAI_API_KEY, config.WEAVIATE_URL, config.RESET_WEAVIATE
    )

openai_service = OpenAIService(config.OPENAI_API_KEY, config)
query_processor = QueryProcessor(openai_service=openai_service)

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"
INFO:root: Weaviate schema is valid: True
INFO:root: Weaviate schema info: Weaviate Schema Information

Class: Product (Entries: 1479)
Properties:
  - Name: name, Type: text, Description: The name of the product.
  - Name: size, Type: text, Description: The size of the product.
  - Name: form, Type: text, Description: The form of the product.
  - Name: processor, Type: text, Description: The processor of the product.
  - Name: core, Type: text, Description: The core of the product.
  - Name: frequency, Type: text, De

===:> Initializing Weaviate


### Helper functions

In [6]:
async def display_search_results(query: str, features: List[str], limit: int = 5):
    print(f"Searching for: {query}")
    results = await weaviate_service.search_products(query, limit=limit)
    print("-" * 80)
    for result in results:
        for feature in features:
            print(f"{str(feature).capitalize()}: {result[feature]}")
    print("-" * 80)
    return results

In [3]:
def get_unique_products(products):
    unique_products = {}
    for prod in products:
        if prod["name"] not in unique_products:
            unique_products[prod["name"]] = Product(
                name=prod["name"],
                summary=prod["summary"],
                form=prod.get("form"),
                io=prod.get("io"),
                manufacturer=prod.get("manufacturer"),
                memory=prod.get("memory"),
                processor=prod.get("processor"),
                size=prod.get("size"),
            )
    return list(unique_products.values())

In [46]:
def compare_products(extended_products: List[str], original_products: List[str]) -> None:
    """Compares products retrieved from original and expanded queries."""
    max_length = max(len(extended_products), len(original_products))

    print(f"{'Original Products':<80}{'Extended Products':<80}")
    print("-" * 160)

    for i in range(max_length):
        original_product = original_products[i] if i < len(original_products) else ""
        extended_product = extended_products[i] if i < len(extended_products) else ""

        print(f"{original_product:<80}{extended_product:<80}")

### Prompt 1


In [10]:
clear_intent_1 = "What are the top Computer on Modules available with high memory and I/O count?"
chat_history = []

#### Original prompt search results

In [11]:
original_products = await display_search_results(clear_intent_1, ["name"], 10)

Searching for: What are the top Computer on Modules available with high memory and I/O count?


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"


--------------------------------------------------------------------------------
Name: Computer On Modules
Name: Not available
Name: Computer On Modules
Name: ET COM Express
Name: TX Computer On Module
Name: Computer On Modules
Name: SOM-5899
Name: Computer On Modules
Name: COMebBD Module
Name: SOM Intel Xeon Processor D COM Express R Type Module
--------------------------------------------------------------------------------


In [12]:
original_products[0]

{'form': 'COMExpress Basic Module',
 'io': 'PCIe x16, PCIe x1, SATAIII, USB 2.0, USB 3.0, GbE, VGA, LVDS, DVI, HDMI, DisplayPort',
 'manufacturer': 'Advantech',
 'memory': 'Dual Channel DDR3L SODIMM sockets, up to 16GB',
 'name': 'Computer On Modules',
 'processor': 'Intel Core i7/i5/i3/Celeron',
 'size': '125 x 95 mm',
 'summary': 'A COMExpress Basic Module with Intel Core i7/i5/i3/Celeron processors, dual channel DDR3L memory support, and extensive I/O options.'}

In [13]:
unique_original_products = get_unique_products(original_products)
unique_original_products

[Product(name='Computer On Modules', summary='A COMExpress Basic Module with Intel Core i7/i5/i3/Celeron processors, dual channel DDR3L memory support, and extensive I/O options.', form='COMExpress Basic Module', io='PCIe x16, PCIe x1, SATAIII, USB 2.0, USB 3.0, GbE, VGA, LVDS, DVI, HDMI, DisplayPort', manufacturer='Advantech', memory='Dual Channel DDR3L SODIMM sockets, up to 16GB', processor='Intel Core i7/i5/i3/Celeron', size='125 x 95 mm'),
 Product(name='Not available', summary='High-performance COM Express Basic Module with Intel Xeon D processors, extensive I/O, and robust thermal management.', form='COM Express Basic Module', io='4 x SATA Ports (6 Gbps), 8 x USB Ports (4 x USB 3.0, 4 x USB 2.0), 1 x LPC, 1 x SPI Bus, 1 x GPIO (8-bit), 1 x Watchdog Timer, 2 x COM Ports, 2 x Ethernet (10GBASE-KR, 1GBASE-T), 32 x PCIe Gen3 lanes', manufacturer='Advantech', memory='DDR4, Max Capacity: 128 GB, Socket: 2 x 260P SODIMM, Dual Channel, ECC/Non-ECC', processor='Intel Xeon D (Ice Lake-D)',

#### Extract product attributes

In [14]:
res, input_tokens, output_tokens = await query_processor.extract_product_attributes(clear_intent_1)
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 76
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 28


Input tokens: 76, Output tokens: 28


{'category': 'Computer on Modules', 'features': ['high memory', 'I/O count']}

#### Generate faceted search parameters

In [15]:
res, input_tokens, output_tokens = await query_processor.generate_faceted_search_params(
    clear_intent_1, ["name", "form", "memory", "io", "processor"]
)
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 76
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 20


Input tokens: 76, Output tokens: 20


{'memory': ['high'], 'io': ['high']}

#### Product reranking

In [16]:
products_for_reranking = [{  "name": p.name,   "summary": p.summary,  "form": p.form,  "io": p.io,  "manufacturer": p.manufacturer,   "memory": p.memory,  "processor": p.processor,  "size": p.size  } for p in unique_original_products]
res, input_tokens, output_tokens = await query_processor.rerank_products(clear_intent_1, products_for_reranking, top_k=10)
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 1373
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 186


rerank_products response from OpenAI: [{'name': 'SOM Intel Xeon Processor D COM Express R Type Module', 'relevance_score': 0.95}, {'name': 'Not available', 'relevance_score': 0.93}, {'name': 'ET COM Express', 'relevance_score': 0.9}, {'name': 'SOM-5899', 'relevance_score': 0.88}, {'name': 'COMebBD Module', 'relevance_score': 0.87}, {'name': 'Computer On Modules', 'relevance_score': 0.85}, {'name': 'TX Computer On Module', 'relevance_score': 0.6}]
Input tokens: 1373, Output tokens: 186


['SOM Intel Xeon Processor D COM Express R Type Module',
 'Not available',
 'ET COM Express',
 'SOM-5899',
 'COMebBD Module',
 'Computer On Modules',
 'TX Computer On Module']

In [17]:
products_for_reranking = [{"name": p.name, "summary": p.summary} for p in unique_original_products]
res, input_tokens, output_tokens = await query_processor.rerank_products(clear_intent_1, products_for_reranking, top_k=10)
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 475
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 186


rerank_products response from OpenAI: [{'name': 'COMebBD Module', 'relevance_score': 0.95}, {'name': 'ET COM Express', 'relevance_score': 0.92}, {'name': 'SOM-5899', 'relevance_score': 0.9}, {'name': 'Computer On Modules', 'relevance_score': 0.88}, {'name': 'SOM Intel Xeon Processor D COM Express R Type Module', 'relevance_score': 0.87}, {'name': 'TX Computer On Module', 'relevance_score': 0.75}, {'name': 'Not available', 'relevance_score': 0.7}]
Input tokens: 475, Output tokens: 186


['COMebBD Module',
 'ET COM Express',
 'SOM-5899',
 'Computer On Modules',
 'SOM Intel Xeon Processor D COM Express R Type Module',
 'TX Computer On Module',
 'Not available']

In [18]:
products_for_reranking = [{"name": p.name, "memory": p.memory, "io": p.io} for p in unique_original_products]
res, input_tokens, output_tokens = await query_processor.rerank_products(
    clear_intent_1, products_for_reranking, top_k=10
)
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 773
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 186


rerank_products response from OpenAI: [{'name': 'SOM Intel Xeon Processor D COM Express R Type Module', 'relevance_score': 0.95}, {'name': 'Not available', 'relevance_score': 0.9}, {'name': 'ET COM Express', 'relevance_score': 0.85}, {'name': 'SOM-5899', 'relevance_score': 0.8}, {'name': 'COMebBD Module', 'relevance_score': 0.75}, {'name': 'Computer On Modules', 'relevance_score': 0.7}, {'name': 'TX Computer On Module', 'relevance_score': 0.5}]
Input tokens: 773, Output tokens: 186


['SOM Intel Xeon Processor D COM Express R Type Module',
 'Not available',
 'ET COM Express',
 'SOM-5899',
 'COMebBD Module',
 'Computer On Modules',
 'TX Computer On Module']

In [19]:
reranked_original_products = res

In [20]:
products_for_reranking = [{"name": p.name} for p in unique_original_products]
res, input_tokens, output_tokens = await query_processor.rerank_products(
    clear_intent_1, products_for_reranking, top_k=10
)
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 210
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 163


rerank_products response from OpenAI: [{'name': 'SOM Intel Xeon Processor D COM Express R Type Module', 'relevance_score': 0.95}, {'name': 'SOM-5899', 'relevance_score': 0.9}, {'name': 'TX Computer On Module', 'relevance_score': 0.85}, {'name': 'ET COM Express', 'relevance_score': 0.8}, {'name': 'COMebBD Module', 'relevance_score': 0.75}, {'name': 'Computer On Modules', 'relevance_score': 0.7}]
Input tokens: 210, Output tokens: 163


['SOM Intel Xeon Processor D COM Express R Type Module',
 'SOM-5899',
 'TX Computer On Module',
 'ET COM Express',
 'COMebBD Module',
 'Computer On Modules']

We get a better product runking when we use in the raking featurs mensioned in generate_faceted_search_params, and extract_product_attributes.

#### Query expansion

In [21]:
res, input_tokens, output_tokens = await query_processor.expand_query(clear_intent_1, [], 5)
print(f'Input tokens: {input_tokens}, Output tokens: {output_tokens}')
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 90
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 95


Input tokens: 90, Output tokens: 95


['What are the top Computer on Modules available with high memory capacity and extensive I/O count?',
 'What are the best Computer on Modules with high memory and a large number of I/O ports?',
 'Which Computer on Modules have the highest memory and the most I/O interfaces?',
 'Top-rated Computer on Modules with maximum memory and high I/O connectivity',
 'Best Computer on Modules featuring high memory and numerous I/O options']

In [19]:
res, input_tokens, output_tokens = await query_processor.expand_query(clear_intent_1, [], 5)
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 90
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 95


Input tokens: 90, Output tokens: 95


['What are the top Computer on Modules available with high memory capacity and extensive I/O count?',
 'What are the best Computer on Modules with large memory and numerous I/O ports?',
 'Which Computer on Modules have the highest memory and the most I/O interfaces?',
 'Top-rated Computer on Modules with maximum memory and high I/O connectivity',
 'Best Computer on Modules featuring high memory and a large number of I/O options']

In [20]:
res, input_tokens, output_tokens = await query_processor.expand_query(clear_intent_1, [], 5, temperature=.7)
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 90
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 87


Input tokens: 90, Output tokens: 87


['Top Computer on Modules with high memory and high I/O count',
 'Best Computer on Modules with large memory capacity and extensive I/O ports',
 'High-performance Computer on Modules featuring substantial memory and numerous I/O interfaces',
 'Leading Computer on Modules with high RAM and multiple input/output options',
 'Top-rated Computer on Modules with significant memory and a high number of I/O connections']

In [11]:
res, input_tokens, output_tokens = await query_processor.expand_query(clear_intent_1, [], 5, temperature=0.7)
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 90
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 87


Input tokens: 90, Output tokens: 87


['Best Computer on Modules with high memory capacity and extensive I/O interfaces',
 'Top-rated Computer on Modules featuring large memory and numerous I/O ports',
 'High-performance Computer on Modules with substantial memory and multiple I/O options',
 'Leading Computer on Modules with high RAM and a wide range of I/O connectivity',
 'Computer on Modules with superior memory specifications and extensive I/O capabilities']

In [21]:
res, input_tokens, output_tokens = await query_processor.expand_query(clear_intent_1, [], 5)
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 90
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 92


Input tokens: 90, Output tokens: 92


['What are the top Computer on Modules available with high memory capacity and extensive I/O count?',
 'What are the best Computer on Modules with large memory and numerous I/O ports?',
 'Which Computer on Modules have the highest memory and the most I/O interfaces?',
 'Top-rated Computer on Modules with maximum memory and high I/O connectivity',
 'Best Computer on Modules featuring high memory and extensive input/output options']

As we can see above the expanded queries variability decreases with the decrease in temperature. This is because the temperature parameter controls the randomness of the output. A higher temperature will result in more randomness and a lower temperature will result in less randomness.

In [22]:
extended_products = []
for query in res:
    products = await display_search_results(query, ["name"])
    extended_products.extend(products)

Searching for: What are the top Computer on Modules available with high memory capacity and extensive I/O count?


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"


--------------------------------------------------------------------------------
Name: Computer On Modules
Name: Not available
Name: Computer On Modules
Name: ET COM Express
Name: TX Computer On Module
--------------------------------------------------------------------------------
Searching for: What are the best Computer on Modules with high memory and a large number of I/O ports?


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"


--------------------------------------------------------------------------------
Name: Computer On Modules
Name: Not available
Name: Computer On Modules
Name: ET COM Express
Name: TX Computer On Module
--------------------------------------------------------------------------------
Searching for: Which Computer on Modules have the highest memory and the most I/O interfaces?


...

--------------------------------------------------------------------------------
Name: Computer On Modules
Name: Computer On Modules
Name: ET COM Express
Name: TX Computer On Module
Name: Computer On Modules
--------------------------------------------------------------------------------
Searching for: Top-rated Computer on Modules with maximum memory and high I/O connectivity


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"


--------------------------------------------------------------------------------
Name: Computer On Modules
Name: Not available
Name: Computer On Modules
Name: TX Computer On Module
Name: SOM-5899
--------------------------------------------------------------------------------
Searching for: Best Computer on Modules featuring high memory and numerous I/O options


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"


--------------------------------------------------------------------------------
Name: Computer On Modules
Name: Not available
Name: Computer On Modules
Name: TX Computer On Module
Name: SOM-5899
--------------------------------------------------------------------------------


In [23]:
unique_extended_products = get_unique_products(extended_products)
unique_extended_products

[Product(name='Computer On Modules', summary='A COMExpress Basic Module with Intel Core i7/i5/i3/Celeron processors, dual channel DDR3L memory support, and extensive I/O options.', form='COMExpress Basic Module', io='PCIe x16, PCIe x1, SATAIII, USB 2.0, USB 3.0, GbE, VGA, LVDS, DVI, HDMI, DisplayPort', manufacturer='Advantech', memory='Dual Channel DDR3L SODIMM sockets, up to 16GB', processor='Intel Core i7/i5/i3/Celeron', size='125 x 95 mm'),
 Product(name='Not available', summary='High-performance COM Express Basic Module with Intel Xeon D processors, extensive I/O, and robust thermal management.', form='COM Express Basic Module', io='4 x SATA Ports (6 Gbps), 8 x USB Ports (4 x USB 3.0, 4 x USB 2.0), 1 x LPC, 1 x SPI Bus, 1 x GPIO (8-bit), 1 x Watchdog Timer, 2 x COM Ports, 2 x Ethernet (10GBASE-KR, 1GBASE-T), 32 x PCIe Gen3 lanes', manufacturer='Advantech', memory='DDR4, Max Capacity: 128 GB, Socket: 2 x 260P SODIMM, Dual Channel, ECC/Non-ECC', processor='Intel Xeon D (Ice Lake-D)',

In [32]:
products_for_reranking = [{"name": p.name, "memory": p.memory, "io": p.io} for p in unique_extended_products]
res, input_tokens, output_tokens = await query_processor.rerank_products(
    clear_intent_1, products_for_reranking, top_k=10
)
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o


INFO:services.openai_service:Input token count: 568
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 128


rerank_products response from OpenAI: [{'name': 'Not available', 'relevance_score': 0.95}, {'name': 'ET COM Express', 'relevance_score': 0.9}, {'name': 'SOM-5899', 'relevance_score': 0.85}, {'name': 'Computer On Modules', 'relevance_score': 0.8}, {'name': 'TX Computer On Module', 'relevance_score': 0.5}]
Input tokens: 568, Output tokens: 128


['Not available',
 'ET COM Express',
 'SOM-5899',
 'Computer On Modules',
 'TX Computer On Module']

In [37]:
compare_products(res, reranked_original_products)

Original Products                                 Extended Products                                 
------------------------------------------------------------
SOM Intel Xeon Processor D COM Express R Type ModuleComputer On Modules                               
Not available                                     Not available                                     
ET COM Express                                    ET COM Express                                    
SOM-5899                                          TX Computer On Module                             
COMebBD Module                                    SOM-5899                                          
Computer On Modules                                                                                 
TX Computer On Module                                                                               


As we can see here we are able to get more product as a result of the expanded query. This is a simple example of how we can use the OpenAI API to expand the query and get more relevant results.

#### Search query

In [38]:
res, input_tokens, output_tokens = await query_processor.generate_search_queries(clear_intent_1, [], 5)
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 85
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 80


Input tokens: 85, Output tokens: 80


['best Computer on Modules with high memory and I/O count',
 'top-rated Computer on Modules with extensive memory and I/O options',
 'high-performance Computer on Modules with large memory and multiple I/O ports',
 'Computer on Modules with maximum memory and I/O capabilities',
 'leading Computer on Modules featuring high memory capacity and numerous I/O interfaces']

In [39]:
res, input_tokens, output_tokens = await query_processor.generate_search_queries(clear_intent_1, [], 5)
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 85
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 76


Input tokens: 85, Output tokens: 76


['best Computer on Modules with high memory and I/O count',
 'top-rated Computer on Modules with extensive memory and I/O options',
 'high memory Computer on Modules with multiple I/O ports',
 'Computer on Modules with maximum memory and I/O capabilities',
 'leading Computer on Modules featuring high memory and numerous I/O interfaces']

In [17]:
res, input_tokens, output_tokens = await query_processor.generate_search_queries(clear_intent_1, [], 5)
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 85
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 69


Input tokens: 85, Output tokens: 69


['best Computer on Modules with high memory',
 'top Computer on Modules with extensive I/O options',
 'high memory Computer on Modules with multiple I/O ports',
 'Computer on Modules with maximum memory and I/O count',
 'leading Computer on Modules with high memory capacity and I/O features']

As can be seen above we have been able to get to consistence in generated search queries using a temprature of 0.1

In [40]:
extended_products = []
for query in res:
    products = await display_search_results(
        query,
        [
            "name"
        ],
    )
    extended_products.extend(products)

Searching for: best Computer on Modules with high memory and I/O count


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"


--------------------------------------------------------------------------------
Name: Not available
Name: Computer On Modules
Name: Computer On Modules
Name: COMebBD Module
Name: TX Computer On Module
--------------------------------------------------------------------------------
Searching for: top-rated Computer on Modules with extensive memory and I/O options


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"


--------------------------------------------------------------------------------
Name: TX Computer On Module
Name: Not available
Name: Computer On Modules
Name: Computer On Modules
Name: SOM-5899
--------------------------------------------------------------------------------
Searching for: high memory Computer on Modules with multiple I/O ports


...

--------------------------------------------------------------------------------
Name: Computer On Modules
Name: Computer On Modules
Name: Not available
Name: Computer On Modules
Name: ET COM Express
--------------------------------------------------------------------------------
Searching for: Computer on Modules with maximum memory and I/O capabilities


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"


--------------------------------------------------------------------------------
Name: Computer On Modules
Name: Computer On Modules
Name: Not available
Name: COMebBD Module
Name: TX Computer On Module
--------------------------------------------------------------------------------
Searching for: leading Computer on Modules featuring high memory and numerous I/O interfaces


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"


--------------------------------------------------------------------------------
Name: Computer On Modules
Name: Not available
Name: TX Computer On Module
Name: Computer On Modules
Name: Computer On Modules
--------------------------------------------------------------------------------


In [41]:
unique_extended_products = get_unique_products(extended_products)
unique_extended_products

[Product(name='Not available', summary='High-performance COM Express Basic Module with Intel Xeon D processors, extensive I/O, and robust thermal management.', form='COM Express Basic Module', io='4 x SATA Ports (6 Gbps), 8 x USB Ports (4 x USB 3.0, 4 x USB 2.0), 1 x LPC, 1 x SPI Bus, 1 x GPIO (8-bit), 1 x Watchdog Timer, 2 x COM Ports, 2 x Ethernet (10GBASE-KR, 1GBASE-T), 32 x PCIe Gen3 lanes', manufacturer='Advantech', memory='DDR4, Max Capacity: 128 GB, Socket: 2 x 260P SODIMM, Dual Channel, ECC/Non-ECC', processor='Intel Xeon D (Ice Lake-D)', size='95 mm x 125 mm'),
 Product(name='Computer On Modules', summary='A high-performance, low-power Computer On Module with 6th Gen Intel Core and Celeron Processors, supporting dual channel DDR4 memory, multiple display outputs, and extensive I/O interfaces.', form='COM Express Compact Module Type 6', io='SATA Ports: 2 (Gen3 6Gb/s, Gen2 3Gb/s), USB Ports: 8 (4 USB 3.0, 4 USB 2.0), PCI Express: 1 x16 PEG, 7 x1 PCIe, HD Audio, LPC, SMBus, I2C B

In [42]:
products_for_reranking = [{"name": p.name, "memory": p.memory, "io": p.io} for p in unique_extended_products]
res, input_tokens, output_tokens = await query_processor.rerank_products(
    clear_intent_1, products_for_reranking, top_k=10
)
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 694
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 153


rerank_products response from OpenAI: [{'name': 'Not available', 'relevance_score': 0.95}, {'name': 'ET COM Express', 'relevance_score': 0.9}, {'name': 'Computer On Modules', 'relevance_score': 0.85}, {'name': 'SOM-5899', 'relevance_score': 0.8}, {'name': 'COMebBD Module', 'relevance_score': 0.75}, {'name': 'TX Computer On Module', 'relevance_score': 0.6}]
Input tokens: 694, Output tokens: 153


['Not available',
 'ET COM Express',
 'Computer On Modules',
 'SOM-5899',
 'COMebBD Module',
 'TX Computer On Module']

In [45]:
reranked_original_products

['SOM Intel Xeon Processor D COM Express R Type Module',
 'Not available',
 'ET COM Express',
 'SOM-5899',
 'COMebBD Module',
 'Computer On Modules',
 'TX Computer On Module']

In [48]:
compare_products(res, reranked_original_products)

Original Products                                                               Extended Products                                                               
------------------------------------------------------------
SOM Intel Xeon Processor D COM Express R Type Module                            Not available                                                                   
Not available                                                                   ET COM Express                                                                  
ET COM Express                                                                  Computer On Modules                                                             
SOM-5899                                                                        SOM-5899                                                                        
COMebBD Module                                                                  COMebBD Module                                        

#### Query Clarification

In [ ]:
res, input_tokens, output_tokens = await query_processor.generate_query_clarification(clear_intent_1, [])
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
res

INFO:services.openai_service:===> Using model: gpt-4o
INFO:services.openai_service:Input token count: 80
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:services.openai_service:Output token count: 45


Input tokens: 80, Output tokens: 45


{'clarification_question': 'Could you please specify the type of applications or industries you are targeting for these Computer on Modules, as well as any specific memory size and I/O count requirements you have in mind?'}